# Table of contents

1. [Import packages and dataset](#Import-packages-and-dataset)
2. [Exploratory Analysis](#Exploratory-analysis)
3. [Data processing](#Data-processing)
4. [ANOVA with respect to age, gender, and region](#ANOVA:-comparing-remote-working-in-age,-gender,-and-region-groups)
5. [Pairwise comparison with respect to age and region](#Tukey's-pairwise-comparison-for-different-age-and-region)
6. [Save the final dataframe](#Save-the-processed,-final-dataframe)

# Import packages and dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scipy.stats as stats
import os
%matplotlib inline  

In [2]:
# Load the data
work_status = pd.read_csv(
    "https://raw.githubusercontent.com/johnjosephhorton/remote_work/master/etl/gcs3rdWave_changedOriginalQ_july4_final.csv", \
    sep=",", 
    header=0,
    )
work_status = work_status.dropna()

# Exploratory analysis

In [3]:
work_status.shape

(25004, 10)

In [4]:
work_status.head(3)

,User ID,Time (UTC),Survey Completion,Publisher Category,Gender,Age,Geography,Weight,Question #1 Answer,Response Time #1 (ms)
0,1000060079,2020-06-27 04:00:00,Complete,News,Female,55-64,US-SOUTH-AL-Pelham,0.922468,I continue to commute to work,23342
1,1000194255,2020-06-27 04:00:00,Complete,News,Female,45-54,US-WEST-NM-Los Lunas,1.110445,"Used to commute, now work from home",20765
2,1000207679,2020-06-26 14:00:00,Complete,News,Female,25-34,US-SOUTH-VA-Washington,1.154895,I continue to commute to work,13883


In [5]:
work_status.describe(exclude = np.number)

,Time (UTC),Survey Completion,Publisher Category,Gender,Age,Geography,Question #1 Answer
count,25004,25004,25004,25004,25004,25004,25004
unique,303,1,4,3,7,823,6
top,2020-06-24 01:00:00,Complete,News,Male,45-54,US-SOUTH-TX,I continue to commute to work
freq,459,25004,23748,11627,4431,747,7845


### Observation: 
- Gender has three groups, one of which is unknown category. 
- Age is recorded in bins. 
- There are 6 options for question #1.

### Check different values for working status survey answers and percentage for each answer.

In [6]:
work_status['Question #1 Answer'].unique()

array(['I continue to commute to work',
       'Used to commute, now work from home',
       'Not working for pay before start of COVID19',
       'I have recently been furloughed or laid-off',
       'Used to work from home and still do',
       'Used to work from home, but now I commute'], dtype=object)

In [7]:
agg_work_status = work_status.groupby('Question #1 Answer').count()[['User ID']].rename(columns={"User ID": "count of users"})

In [8]:
agg_work_status['percentage'] = agg_work_status[['count of users']]*100 / work_status.shape[0]

In [9]:
agg_work_status

,count of users,percentage
Question #1 Answer,,
I continue to commute to work,7845,31.374980
I have recently been furloughed or laid-off,1786,7.142857
Not working for pay before start of COVID19,6699,26.791713
"Used to commute, now work from home",4903,19.608863
Used to work from home and still do,2739,10.954247
"Used to work from home, but now I commute",1032,4.127340


### Observation: Nearly 20% of the workers switch to work from home from commuting. 

# Data processing

### Extract state and region information from geography column

In [10]:
# convert column dtype from object to string 
work_status['Geography'] = work_status['Geography'].astype("string")

work_status['state'] = work_status['Geography'].str.split("-", n = 3, expand = True).get(2).str.replace("'", '')

In [11]:
work_status['region'] = work_status['Geography'].str.split("-", n = 3, expand = True).get(1)

### Check the number of samples for each category of state, region, age, and gender

In [12]:
work_status.groupby('state').count()['User ID']

state
AK      34
AL     523
AR      63
AZ     632
CA    1710
CO     824
CT     305
DC      77
DE      70
FL     956
GA     988
HI      32
IA     511
ID     166
IL    1853
IN     513
KS     169
KY     165
LA     170
MA     225
MD     220
ME      71
MI     332
MN     832
MO     782
MS     252
MT     161
NC    1347
ND      88
NE     305
NH      25
NJ     597
NM     303
NV     265
NY    1153
OH     571
OK     426
OR     351
PA     940
RI      23
SC     336
SD     128
TN     346
TX    1882
UT     385
VA    1332
VT      46
WA     422
WI     957
WV      18
WY      93
Name: User ID, dtype: int64

In [13]:
work_status.groupby('region').count()['User ID']

region
MIDWEST      7041
NORTHEAST    3385
SOUTH        9171
WEST         5378
Name: User ID, dtype: int64

In [14]:
work_status.groupby('Age').count()[['User ID']]

,User ID
Age,
18-24,1581
25-34,4212
35-44,4121
45-54,4431
55-64,4356
65+,3851
Unknown,2452


In [15]:
work_status.groupby('Gender').count()[['User ID']]

,User ID
Gender,
Female,11083
Male,11627
Unknown,2294


### Create a column indicating whether the person is remote working

In [16]:
work_status['remote'] = work_status['Question #1 Answer']\
                        .apply(lambda x: 1 if x == "Used to commute, now work from home" else 0)

### Check the distribution of age for people who remain working from home since pandemic

In [17]:
work_status[work_status['Question #1 Answer'] == 'Used to work from home and still do'].groupby('Age').count()[['User ID']]

,User ID
Age,
18-24,143
25-34,386
35-44,485
45-54,499
55-64,571
65+,389
Unknown,266


### Check the distribution of answers for people over 65 years old

In [18]:
work_status[work_status['Age'] == '65+'].groupby('Question #1 Answer').count()[['User ID']]

,User ID
Question #1 Answer,
I continue to commute to work,853
I have recently been furloughed or laid-off,195
Not working for pay before start of COVID19,1813
"Used to commute, now work from home",495
Used to work from home and still do,389
"Used to work from home, but now I commute",106


### Remove unknown group regards to age, gender, and region variables 

In [19]:
age_remove_na = work_status[work_status['Age'] != 'Unknown']

In [20]:
gender_remove_na = work_status[work_status['Gender'] != 'Unknown']

In [21]:
region_remove_na = work_status.dropna()

# ANOVA: comparing remote working in age, gender, and region groups

In [22]:
model_age = ols('remote ~ C(Age)', data=age_remove_na).fit()
aov_table_age = sm.stats.anova_lm(model_age, typ=2)
aov_table_age

,sum_sq,df,F,PR(>F)
C(Age),41.223066,5.0,52.299008,3.911905e-54
Residual,3554.236627,22546.0,NaN,NaN


### Observation: There is a significant difference between percentage of people who switched to remote for different age groups.

In [23]:
model_gender = ols('remote ~ C(Gender)', data=gender_remove_na).fit()
aov_table_gender = sm.stats.anova_lm(model_gender, typ=2)
aov_table_gender

,sum_sq,df,F,PR(>F)
C(Gender),0.565069,1.0,3.545408,0.059723
Residual,3619.211638,22708.0,NaN,NaN


### Observation: There is not a significant difference between percentage of people who switched to remote for gender.

In [24]:
stats.levene(region_remove_na['remote'][region_remove_na['region'] == 'SOUTH'],
             region_remove_na['remote'][region_remove_na['region'] == 'WEST'],
             region_remove_na['remote'][region_remove_na['region'] == 'MIDWEST'],
             region_remove_na['remote'][region_remove_na['region'] == 'NORTHEAST'])

LeveneResult(statistic=9.151491636424833, pvalue=4.760629756513708e-06)

### Observation: There is a significant difference between percentage of people switches to remote working between different regions.

In [30]:
model_gender_age = ols('remote ~ C(Gender) * C(Age)', data=gender_remove_na).fit()
aov_table_gender_age = sm.stats.anova_lm(model_gender_age, typ=2)
aov_table_gender_age

,sum_sq,df,F,PR(>F)
C(Gender),1.099867,1.0,6.985810,8.221489e-03
C(Age),41.771080,6.0,44.218191,4.612748e-54
C(Gender):C(Age),4.113143,6.0,4.354107,2.122036e-04
Residual,3573.327414,22696.0,NaN,NaN


### Observation: There is a significant interaction effect between age and gender variable. 

# Tukey's pairwise comparison for different age and region

In [25]:
age_compare = pairwise_tukeyhsd(endog=age_remove_na['remote'], groups=age_remove_na['Age'], alpha=0.05)
print(age_compare)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
 18-24  25-34   0.0472  0.001  0.0138  0.0806   True
 18-24  35-44   0.0515  0.001   0.018   0.085   True
 18-24  45-54   0.0275 0.1695 -0.0057  0.0606  False
 18-24  55-64  -0.0282 0.1496 -0.0614   0.005  False
 18-24    65+  -0.0631  0.001 -0.0969 -0.0293   True
 25-34  35-44   0.0043    0.9 -0.0205  0.0291  False
 25-34  45-54  -0.0197 0.1914 -0.0441  0.0046  False
 25-34  55-64  -0.0754  0.001 -0.0998 -0.0509   True
 25-34    65+  -0.1103  0.001 -0.1355 -0.0851   True
 35-44  45-54   -0.024 0.0585 -0.0485  0.0005  False
 35-44  55-64  -0.0797  0.001 -0.1043 -0.0551   True
 35-44    65+  -0.1146  0.001   -0.14 -0.0892   True
 45-54  55-64  -0.0557  0.001 -0.0798 -0.0315   True
 45-54    65+  -0.0906  0.001 -0.1155 -0.0657   True
 55-64    65+  -0.0349  0.001 -0.0599 -0.0099   True
----------------------------------------------

In [26]:
region_compare = pairwise_tukeyhsd(endog=region_remove_na['remote'], groups=region_remove_na['region'], alpha=0.05)
print(region_compare)

   Multiple Comparison of Means - Tukey HSD, FWER=0.05    
  group1    group2  meandiff p-adj   lower   upper  reject
----------------------------------------------------------
  MIDWEST NORTHEAST   0.0182 0.1247 -0.0031  0.0395  False
  MIDWEST     SOUTH  -0.0183 0.0188 -0.0345 -0.0022   True
  MIDWEST      WEST  -0.0183 0.0533 -0.0368  0.0002  False
NORTHEAST     SOUTH  -0.0365  0.001  -0.057  -0.016   True
NORTHEAST      WEST  -0.0365  0.001 -0.0589 -0.0141   True
    SOUTH      WEST      0.0    0.9 -0.0175  0.0175  False
----------------------------------------------------------


# Save the processed, final dataframe

In [27]:
new_df_path = os.path.join(os.getcwd(), 'data', 'final_df.csv')
new_df_path

'/Users/apple/Desktop/Fall_2020/data-512/data-512-final/data/final_df.csv'

In [28]:
work_status.to_csv(new_df_path, index=False)